# Connecting to Riak

Now, let's start the Python and get set up

In [4]:
import riak

If you are using a single local Riak node, use the following to create a new client instance:

In [5]:
myClient = riak.RiakClient()
myClient.ping()

True

# Creating Objects In Riak

First, let’s create a few objects and a bucket to keep them in.

In [6]:
myBucket = myClient.bucket('test')

val1 = 1
key1 = myBucket.new('one', data=val1)
key1.store()

In this first example, we have stored the integer 1 with the lookup key of one. Next let’s store a simple string value of two with a matching key.

In [7]:
val2 = "two"
key2 = myBucket.new('two', data=val2)
key2.store()

That was easy. Finally, let’s store a bit of JSON. You will probably recognize the pattern by now.

In [8]:
val3 = {"myValue": 3}
key3 = myBucket.new('three', data=val3)
key3.store()

# Reading Objects From Riak

Now that we have a few objects stored, let’s retrieve them and make sure they contain the values we expect.

In [10]:
fetched1 = myBucket.get('one').data
fetched2 = myBucket.get('two').data
fetched3 = myBucket.get('three').data

print "one: ", fetched1
print "two: ", fetched2
print "three: ", fetched3


one:  1
two:  two
three:  {u'myValue': 3}


That was easy. We simply request the objects by key.

In [11]:
curl http://localhost:8098/riak/test/one

1

In [12]:
curl http://localhost:8098/riak/test/two

"two"

In [13]:

curl http://localhost:8098/riak/test/three

{"myValue": 3}

# Updating Objects In Riak

While some data may be static, other forms of data may need to be updated. This is also easy to accomplish. Let’s update the value of myValue in the 3rd example to 42.

In [36]:
fetched3.data["myValue"] = 42
fetched3.store()

In [37]:
! curl http://localhost:8098/riak/test/three

{"myValue": 42}

# Deleting Objects From Riak

Nothing is complete without a delete. Fortunately, that's easy too.

In [13]:
fetched1.delete()
fetched2.delete()
fetched3.delete()

Now we can verify that the objects have been removed from Riak.

In [14]:
assert myBucket.get('one').exists == False
assert myBucket.get('two').exists == False
assert myBucket.get('three').exists == False

# Working With Complex Objects

Since the world is a little more complicated than simple integers and bits of strings, let’s see how we can work with more complex objects. Take for example, this object that encapsulates some knowledge about a book.

In [15]:
book = {
  'isbn': "1111979723",
  'title': "Moby Dick",
  'author': "Herman Melville",
  'body': "Call me Ishmael. Some years ago...",
  'copies_owned': 3
}

booksBucket = myClient.bucket('books')
newBook = booksBucket.new(book['isbn'], data=book)
newBook.store()

All right, so we have some information about our Moby Dick collection that we want to save. Storing this to Riak should look familiar by now:

In [16]:
booksBucket = myClient.bucket('books')
newBook = booksBucket.new(book['isbn'], data=book)
newBook.store()

Some of you may be thinking, “But how does the Python Riak client encode/decode my object?” If we fetch our book back and print the raw encoded data, we shall know:

In [17]:
fetchedBook = booksBucket.get(book['isbn'])

print(fetchedBook.encoded_data)

{"body": "Call me Ishmael. Some years ago...", "author": "Herman Melville", "isbn": "1111979723", "copies_owned": 3, "title": "Moby Dick"}


**JSON!** The Riak Python client library encodes things as JSON when it can.

In [18]:
{"body": "Call me Ishmael. Some years ago...",
"author": "Herman Melville", "isbn": "1111979723",
"copies_owned": 3, "title": "Moby Dick"}

{'author': 'Herman Melville',
 'body': 'Call me Ishmael. Some years ago...',
 'copies_owned': 3,
 'isbn': '1111979723',
 'title': 'Moby Dick'}

Finally, let’s clean up our mess:

In [19]:
fetchedBook.delete()

# Storing binary data

Riak can also handle binary data, such as images, audio files, etc. Storing binary data looks almost identical:

In [5]:
book_photo_bucket = myClient.bucket('book_photo')

In [6]:
book_photo_bucket = myClient.bucket('book_photo')

photo = book_photo_bucket.new_from_file('1111979723', './images/moby_dick_book_cover.jpg')
photo.store()

In [7]:
photo = book_photo_bucket.get('1111979723')
photo_data = photo.encoded_data

print type(photo_data)

<type 'str'>


In [8]:
print photo_data[:100]

���� Exif  II*            �� Ducky     P  �� &Adobe d�    
 L� . � ��� � 


In [9]:
! curl -s http://localhost:8098/riak/book_photo/1111979723 | hexdump | head 

0000000 d8ff e1ff 1800 7845 6669 0000 4949 002a
0000010 0008 0000 0000 0000 0000 0000 ecff 1100
0000020 7544 6b63 0079 0001 0004 0000 0050 ff00
0000030 00ee 4126 6f64 6562 6400 00c0 0000 0301
0000040 1500 0304 0a06 000d 4c03 008a 2e04 0016
0000050 0c07 00bf fb0b ff15 00db 0084 0202 0202
0000060 0202 0202 0202 0203 0202 0403 0203 0302
0000070 0504 0404 0404 0504 0506 0505 0505 0605
0000080 0706 0807 0707 0906 0a09 090a 0c09 0c0c
0000090 0c0c 0c0c 0c0c 0c0c 0c0c 0c0c 0301 0303


In [47]:
photo.delete()

In [10]:
! curl -i http://localhost:8098/riak/book_photo/1111979723

HTTP/1.1 200 OK
X-Riak-Vclock: a85hYGBgzGDKBVI8ypz/fm5oDu5lYFAXzGBKZMxjZVBVW3yeLwsA
Vary: Accept-Encoding
Server: MochiWeb/1.1 WebMachine/1.10.8 (that head fake, tho)
Link: </riak/book_photo>; rel="up"
Last-Modified: Mon, 26 Oct 2015 22:33:09 GMT
ETag: "5ECCiYtX3q62WVBlTTJ0LN"
Date: Mon, 26 Oct 2015 22:33:31 GMT
Content-Type: image/jpeg
Content-Length: 785175

���� Exif  II*            �� Ducky     P  �� &Adobe d�    
 L� . � ��� � 		

�� 
� ��            	              0!1@P`"pA2#4356$��B%&�� 
��Qj+��4��"��L�T��f4��e!���Kwg:sfGY�okM��9
����I*�2�y|����#V�S<����:Q�%2�(�W$�D���cb����F����[�AFXUEI�L�2D5�DD����i��ofq�g�Z�����a��|��kSB����Me��m���>^\[��M�M�\��"7KP�IcVMI�����4Ss[[��f��8��Fd�0������JP)T��*R�.��������o�ST'.1����^doH@M&���W����UN���M)�\Yu����7�vMiZl���X���XV����j�+�#Q\�o&�N